## Our steps
- Initialize weights randomly
- Give the model a char pair (input char & target char. The target char is the char the network should guess, its the next char in our sequence)
- Forward pass (We calculate the probability for every possible next char according to the state of the model, using the paramters)
- Measure error (the distance between the previous probability and the target char)
- We calculate gradients for each of our parameters to see their impact they have on the loss (backpropagation through time)
- update all parameters in the direction via gradients that help to minimise the loss
- Repeat! Until our loss is small AF

## The code contains 4 parts
- Load the trainning data
    - encode char into vectors
- Define the Recurrent Network
- Define a loss function
    - Forward pass
    - Loss
    - Backward pass
- Define a function to create sentences from the model
- Train the network
    - Feed the network
    - Calculate gradient and update the model parameters
    - Output a text to see the progress of the training
    
    
## Load the training data
The network need a big txt file as an input.

The content of the file will be used to train the network.

I use Methamorphosis from Kafka (Public Domain). 

Because Kafka was one weird dude. I like.

In [5]:
data = open('kafka.txt', 'r').read()

chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

print ("Length: {}, unique: {}".format(data_size, vocab_size))

Length: 137629, unique: 81


## Encode/Decode char/vector
Neural networks operate on vectors (a vector is an array of float)

So we need a way to encode and decode a char as a vector.
We'll count the number of unique chars (vocab_size). 

That will be the size of the vector. The vector contains only zero exept for the position of the char wherae the value is 1.

In [8]:
char_to_ix = { ch:ix for ix, ch in enumerate(chars)}
ix_to_char = { ix:ch for ix, ch in enumerate(chars)}

print(char_to_ix)
print(ix_to_char)

{'E': 0, 'V': 1, 'a': 2, 't': 3, 'C': 4, '\n': 5, 'i': 6, 'B': 7, 'P': 8, 'U': 9, 'Y': 10, 'M': 11, 'R': 12, '6': 13, 'x': 14, '0': 15, "'": 16, '?': 17, '5': 18, '2': 19, '1': 20, 'o': 21, 'Ã': 22, 'N': 23, 'X': 24, '%': 25, '@': 26, 'z': 27, '(': 28, ' ': 29, 'u': 30, 'w': 31, 'e': 32, 'S': 33, 'J': 34, 'v': 35, 'm': 36, 'G': 37, 'y': 38, '7': 39, 'H': 40, 'W': 41, '4': 42, 'f': 43, 'A': 44, 'F': 45, 'p': 46, 'b': 47, 'T': 48, ',': 49, '9': 50, 'j': 51, '§': 52, '*': 53, 'Q': 54, '.': 55, 'O': 56, '-': 57, 's': 58, '!': 59, 'n': 60, 'I': 61, 'h': 62, 'D': 63, '/': 64, ')': 65, ':': 66, 'k': 67, 'g': 68, 'c': 69, '$': 70, 'l': 71, 'q': 72, '3': 73, 'r': 74, 'K': 75, 'd': 76, '8': 77, '"': 78, ';': 79, 'L': 80}
{0: 'E', 1: 'V', 2: 'a', 3: 't', 4: 'C', 5: '\n', 6: 'i', 7: 'B', 8: 'P', 9: 'U', 10: 'Y', 11: 'M', 12: 'R', 13: '6', 14: 'x', 15: '0', 16: "'", 17: '?', 18: '5', 19: '2', 20: '1', 21: 'o', 22: 'Ã', 23: 'N', 24: 'X', 25: '%', 26: '@', 27: 'z', 28: '(', 29: ' ', 30: 'u', 31: 'w',

## Finaly we create a vector from a char like this:
The dictionary defined above allosw us to create a vector of size 61 instead of 256.

Here and exemple of the char 'a'

The vector contains only zeros, except at position char_to_ix['a'] where we put a 1.

In [10]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['A']] = 1

print(vector_for_char_a.ravel())

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.]


## Definition of the network
The neural network is made of 3 layers:
- an input layer
- an hidden layer
- an output layer
All layers are fully connected to the next one: each node of a layer are conected to all nodes of the next layer.
    
The hidden layer is connected to the output and to itself: the values from an iteration are used for the next one.
    
To centralise values that matter for the training (hyper parameters) we also define the sequence lenght and the learning rate

In [26]:
# hyperparameters
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

The model parameters are adjusted during the trainning.
- Wxh are parameters to connect a vector that contain one input to the hidden layer.
- Whh are parameters to connect the hidden layer to itself. This is the Key of the Rnn: Recursion is done by injecting the previous values from the output of the hidden state, to itself at the next iteration.
- Why are parameters to connect the hidden layer to the output
- bh contains the hidden bias
- by contains the output bias
You'll see in the next section how theses parameters are used to create a sentence.

In [27]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size) * 0.1
Whh = np.random.randn(hidden_size, hidden_size) * 0.1
Why = np.random.randn(vocab_size, hidden_size) * 0.1
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

## Define the loss function
The loss is a key concept in all neural networks training. It is a value that describe how good is our model.
The smaller the loss, the better our model is.
(A good model is a model where the predicted output is close to the training output)
During the training phase we want to minimize the loss.

##### The loss function calculates the loss but also the gradients (see backward pass):
- It perform a forward pass: calculate the next char given a char from the training set.
- It calculate the loss by comparing the predicted char to the target char. (The target char is the input following char in the tranning set)
- It calculate the backward pass to calculate the gradients

##### This function take as input:

- a list of input char
- a list of target char
- and the previous hidden state

##### This function outputs:

- the loss
- the gradient for each parameters between layers
- the last hidden state

In [39]:
def lossFun(inputs, targets, hprev): 
    xs, hs, ys, ps = {}, {}, {}, {}
    
    hs[-1] = np.copy(hprev)
    
    #initialize loss to 0
    loss = 0
    
    #forward pass
                                                                                      
    for t in range(len(inputs)):
        # encode in 1-of-k representation (we place a 0 vector as the t-th input)   
        xs[t] = np.zeros((vocab_size,1))    
        # Inside that t-th input we use the integer in "inputs" list to  set the correct
        xs[t][inputs[t]] = 1 
        # hidden state 
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)   
        # unnormalized log probabilities for next chars
        ys[t] = np.dot(Why, hs[t]) + by   
        # probabilities for next chars 
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) 
        # softmax (cross-entropy loss) 
        loss += -np.log(ps[t][targets[t],0])  
    # backward pass: compute gradients going backwards    
      #initalize vectors for gradient values for each set of weights 
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        #output probabilities
        dy = np.copy(ps[t])
        #derive our first gradient
        dy[targets[t]] -= 1 # backprop into y  
        #compute output gradient -  output times hidden states transpose
        #When we apply the transpose weight matrix,  
        #we can think intuitively of this as moving the error backward
        #through the network, giving us some sort of measure of the error 
        #at the output of the lth layer. 
        #output gradient
        dWhy += np.dot(dy, hs[t].T)
        #derivative of output bias
        dby += dy
        #backpropagate!
        dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
        dbh += dhraw #derivative of hidden bias
        dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
        dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
        dhnext = np.dot(Whh.T, dhraw) 
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients  
        
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

## Create a sentence from the model

In [40]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
    """                                                                                                                                                                                         
    sample a sequence of integers from the model                                                                                                                                                
    h is memory state, seed_ix is seed letter for first time step   
    n is how many characters to predict
    """
    #create vector
    x = np.zeros((vocab_size, 1))
    #customize it for our seed char
    x[seed_ix] = 1
    #list to store generated chars
    ixes = []
    #for as many characters as we want to generate
    for t in range(n):
        #a hidden state at a given time step is a function 
        #of the input at the same time step modified by a weight matrix 
        #added to the hidden state of the previous time step 
        #multiplied by its own hidden state to hidden state matrix.
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        #compute output (unnormalised)
        y = np.dot(Why, h) + by
        ## probabilities for next chars
        p = np.exp(y) / np.sum(np.exp(y))
        #pick one with the highest probability 
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        #create a vector
        x = np.zeros((vocab_size, 1))
        #customize it for the predicted char
        x[ix] = 1
        #add it to the list
        ixes.append(ix)

    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print ('----\n {} \n----'.format(txt, ))

hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)

----
 ,03Ohk/
Pq9UyLEo.wR3*V
YH,eX(U§zK)VsdQiFki uT!QF8x2OQAWLFXW0:)e2'@Qt.P$" ::fi?766wu8"r@L§YXA2GB6LK§whR:8q8icy)t:5Bfxls"z4OXMLdP :"lMQyX0PE3/k'i3WJkf(4F1rQgWK6§5Ihm91(NVweG:UT/)044rJBz;!,%%zuufgk3'y)t) 
----


In [41]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets", targets)

inputs [56, 60, 32, 29, 36, 21, 74, 60, 6, 60, 68, 49, 29, 31, 62, 32, 60, 29, 37, 74, 32, 68, 21, 74, 29]
targets [60, 32, 29, 36, 21, 74, 60, 6, 60, 68, 49, 29, 31, 62, 32, 60, 29, 37, 74, 32, 68, 21, 74, 29, 33]


In [42]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    # check "How to feed the loss function to see how this part works
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
        p = 0 # go from start of data                                                                                                                                                             
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

    # sample from the model now and then                                                                                                                                                        
    if n % 1000 == 0:
        print ('iter {}, loss: {}'.format(n, smooth_loss)) # print progress
        sample(hprev, inputs[0], 200)

    # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seq_length # move data pointer                                                                                                                                                         
    n += 1 # iteration counter

iter 0, loss: 109.86146025086015
----
 n CI§2V7Y5j0w6BbI8fV4Ymft0BKxHw;3?S!XHkdRER"yfz6l-bwaQj-wj5H
§yA3;s8f7RYlp2laFYcV06
g9@QeI8yLuCv.kN8?6
@Ãe*"hBp./nc"tItl*Q@,I7RD!-gV"u3o8@cK$o
UB;$nOp2y;Spa
YY820$,o9:7fT6 "VDe(hqU%c37R7poM,eyH;)Qq1,a 
----
iter 1000, loss: 87.40981938408657
----
  thonsd hhiy nssmr he pet sim
 aate "R amltie t hat oy n no thos nt hisp habid tpfgn redoly ps,reugrudistronsg tS rhe teua. frolt tuen, fme -, au hcege fen txtecuyo lovam aacte%iplun e inie sohnaauHem 
----
iter 2000, loss: 70.23265935417703
----
 ow, aimrele wen che, e etiw ras theulg. Ntherlme y ilfmed pet the twe wathe the inmew mat wee fSed mirouthe o feroad ogo tus alen hant balerecos wrrotheturged ay anwame doud aul srily fhit heas that e 
----
iter 3000, loss: 61.11582728807139
----
 ll Gr walls. "torais the pot fo him lpache sullt kaspy sot, wat couln; wend ralf worit s the sory shirved wem to al yor. juad sidl thetW anor feengher larsirtund, od ath tohr ore.r.
TX- wad whe.
?Tinf 
----
iter 4000,

iter 33000, loss: 48.74249857293732
----
 bed com the wat wneare outer werny
daillicp he se the ousll.

Heazre
niged of the plod nectinellyy
- midcomatt out the liking sid andu.  Mabl.  !900Vrocerth the Projse toddimistrot ention, wive contlu 
----
iter 34000, loss: 47.19072763833095
----
 d wisterce he door.

Her the was meatly roon growing, hawf on bepent his fathe) it qustom. Mrt qunvert kive ppened in baacelfy though the of tho det sfoat hid he a cakes of the, fall and his payisict  
----
iter 35000, loss: 44.79448606399877
----
 s. nis doof s iful toing wource. uh quit bundely deally promay tomen, theice hard steemechand atibed thatu onemert was evre even of keatil arore ive to thending. Ibe fle shat Them, and buet and his ,  
----
iter 36000, loss: 43.39700818616252
----
 eled, painll qulit seaps to s ither un hamvey." ghe deathen it; tay te. He couther nesst coment noweis to his saving and Gregor to mosher that heud with these pale tume that anrsseenfar would sfother  
----
iter

iter 66000, loss: 45.182569675542595
----
 unded perveryed you takes and pleif
eefulais on the paitay turing to caPmonting
lock prively of bealccleave all hive Poveroonss to flit stalcanbergonfound cid prepers sects atather'che
Projet wive nov 
----
iter 67000, loss: 44.41079211535305
----
 ho deoscing dond in for cofcersting the downudgntionelar to your go, open roome seear withoc forCalless intlee for with whice jusw and clomer of the he oncher whan your Seed of and outnghst all, jug;  
----
iter 68000, loss: 42.33987882479452
----
 ndach did had ash sundurly perister the reest not simith, sa%. Gregored all hamp hce go used, he he could thad bentiod. Ding of stots she she romearded to sleading a sase ad as fabit, said.  Greger hi 
----
iter 69000, loss: 41.1719878773086
----
 istery by when foomt hat dewwan the mive intlist it houn had sed over fome it paunked avo buct, Grejust but all, caaitely on onod the foll would in, fortainsion weren ass been dare the lainfy's they l 
----
iter

iter 99000, loss: 43.34136748015422
----
 ed
way to mupelS "IAMEDIThIINTSINCICEISEVE PRENTCO ENT/ECT OR Y0RAGuted a pliap "hich us of worked, parite pata?

"Sil
wothin. BAbeced tingenth abyov mupeter. So Fou contmoting be this with a pally or 
----
iter 100000, loss: 43.01869338532358
----
 des to the foot that welt being loned cacent infoyh Gother any letad was ancores forbid clonging had farm and bechen fide, sty, dungo any in alid d as Gregor for go dive begraads no; af it wealf 1./
P 
----
